<a href="https://colab.research.google.com/github/av-gav/itmo-nlp/blob/main/Task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wikipedia
!pip install tatoebatools
from tatoebatools import tatoeba
import wikipedia
from bs4 import BeautifulSoup
import requests
import time
import re
import pandas as pd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11696 sha256=902a898b925fa49883f3ab7fad1455ee363734436882712764aac3ad88f5d785
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for SQLAlchemy: filename=SQLAlchemy-1.4.23-cp310-cp310-linux_x86_64.whl size=1554857 sha256=4d4eb88349753c8d02470b07d9c46f5246643045bd47a7ac99819aa29186256c
  Stored in directory: /root/.cache/pip/wheels/b1/83/74/6276352424e42f57b3878fb724f70cbd684af64bd4f5f5a04f
Successfully built SQLAlchemy
  Attempting uninstall: SQLAlchemy
    

In [35]:
# List of languages with cyrillic script, for which there are texts in Wikipedia, Tatoeba and UDHR

lang_str = '''Abkhazian	ab	abk	abk
Adyghe	ady	ady	ady
Altai	alt	alt	alt
Avaric	av	ava	
Bashkir	ba	bak	
Belarusian	be	bel	bel
Bulgarian	bg	bul	bul
Buriat	bxr	bua	
Chechen	ce	che	
Chuvash	cv	chv	
Eastern Mari	mhr	mhr	
Erzya	myv	myv	
Ingush	inh	inh	
Kabardian	kbd	kbd	kbd
Kalmyk	xal	xal	
Karachay-Balkar	krc	krc	
Kazakh	kk	kaz	kaz
Komi	kv	kpv	
Komi-Permyak	koi	koi	koi
Kyrgyz	ky	kir	kir
Lak	lbe	lbe	
Lezghian	lez	lez	
Macedonian	mk	mkd	mkd
Moksha	mdf	mdf	
Mongolian	mn	mon	khk
Ossetic	os	oss	oss
Russian	ru	rus	rus
Rusyn	rue	rue	
Serbian	sr	srp	srp_cyrl
Tajik	tg	tgk	tgk
Tatar	tt	tat	tat
Tuvinian	tyv	tyv	tyv
Udmurt	udm	udm	
Ukrainian	uk	ukr	ukr
Western Mari	mrj	mrj	
Yakut	sah	sah	sah'''

lang_data = {y[0]:{"wiki":y[1], "tato":y[2], "udhr":y[3], "label": i} for i,y in enumerate([x.split('\t') for x in lang_str.split('\n')])}
lang_data

{'Abkhazian': {'wiki': 'ab', 'tato': 'abk', 'udhr': 'abk', 'label': 0},
 'Adyghe': {'wiki': 'ady', 'tato': 'ady', 'udhr': 'ady', 'label': 1},
 'Altai': {'wiki': 'alt', 'tato': 'alt', 'udhr': 'alt', 'label': 2},
 'Avaric': {'wiki': 'av', 'tato': 'ava', 'udhr': '', 'label': 3},
 'Bashkir': {'wiki': 'ba', 'tato': 'bak', 'udhr': '', 'label': 4},
 'Belarusian': {'wiki': 'be', 'tato': 'bel', 'udhr': 'bel', 'label': 5},
 'Bulgarian': {'wiki': 'bg', 'tato': 'bul', 'udhr': 'bul', 'label': 6},
 'Buriat': {'wiki': 'bxr', 'tato': 'bua', 'udhr': '', 'label': 7},
 'Chechen': {'wiki': 'ce', 'tato': 'che', 'udhr': '', 'label': 8},
 'Chuvash': {'wiki': 'cv', 'tato': 'chv', 'udhr': '', 'label': 9},
 'Eastern Mari': {'wiki': 'mhr', 'tato': 'mhr', 'udhr': '', 'label': 10},
 'Erzya': {'wiki': 'myv', 'tato': 'myv', 'udhr': '', 'label': 11},
 'Ingush': {'wiki': 'inh', 'tato': 'inh', 'udhr': '', 'label': 12},
 'Kabardian': {'wiki': 'kbd', 'tato': 'kbd', 'udhr': 'kbd', 'label': 13},
 'Kalmyk': {'wiki': 'xal', 

In [47]:
# get links to receive data from Wikipedia

hrefs = {}
wiki_langs = [v['wiki'] for v in lang_data.values()]
wiki_pages = ['Che Guevara', 'Sport', 'Computer', 'Europe', 'China', 'Year', 'History', 'Metal', 'Japan', 'City', 'Music', 'Rome', 'Earth', 'Milk', 'Mathematics', 'Air', 'South America', 'Dog', 'Bulgaria', 'Sea', 'Money', 'River', 'Egypt', 'Water', 'Tiger', 'Oxygen', 'Rain', 'United Nations', 'Science', 'Star', 'Geography', 'Food', 'Nicaragua', 'Leonardo da Vinci', 'Dmitry Gulia', 'Circassians', 'Association football', 'Italy', 'Heart', 'Time', 'Bible', 'Atom', 'Art', 'Sun', 'Human', 'Cyprus', 'New York City', 'Culture', 'Moon', 'Wikipedia', 'North America', 'Islam', 'Color', 'Animal', 'Ocean', 'Fire', 'Gold', 'Planet', 'Bird', 'Cat', 'Christianity', 'Paris', 'Belarus', 'Eye', 'Month', 'Africa', 'Plant', 'Physics', 'Tree', 'Mountain', 'Fish', 'Number', 'Finland', 'Bread', 'India', 'Language', 'Continent', 'Ukraine', 'Education', 'Russia', 'Internet', 'Iron', 'Asia', '20th century', 'Religion']
for page in wiki_pages:
  print(page, end=' ')
  html = requests.get(f'https://en.wikipedia.org/wiki/{page}')
  soup = BeautifulSoup(html.text, 'html.parser')
  links = soup.select("#p-lang-btn a.interlanguage-link-target")
  for a in links:
    lang = a['hreflang']
    if lang in wiki_langs:
      if lang not in hrefs:
        hrefs[lang] = []
      hrefs[lang] += [requests.utils.unquote(a['href'])]
      print(".", end='')
      time.sleep(.05)
  print()

Che Guevara ......................
Sport .......................
Computer ........................
Europe ..................................
China ..................................
Year ............................
History ...........................
Metal .................
Japan ....................................
City .............................
Music ......................
Rome .............................
Earth ...................................
Milk ........................
Mathematics ........................
Air ..............
South America ............................
Dog ................................
Bulgaria ..................................
Sea ..............................
Money ..........................
River ..........................
Egypt ...............................
Water .........................
Tiger .......................
Oxygen ..........................
Rain .......................
United Nations .............................
Science .............

In [48]:
cyrillic = "".join([chr(x) for x in range(0x0400, 0x0460)] + [chr(x) for x in range(0x048A, 0x0500)])

def check_cyrillic(word):   # return true if all symbols in the string are cyrillic or non alphabetic
  for ch in word:
    if ch not in cyrillic and ch.isalpha():   # alphabet chars not from cyriliic table are not allowed
      return False
  return True

def handle_link(link):
  article = requests.get(link)
  soup = BeautifulSoup(article.text, 'html.parser')
  contents = soup.select("p")
  texts = []
  for p in contents:
    text = p.get_text()
    t = re.sub(r' \([^)]*\)| \[[^\]]*\]| «[^»]*»', '', text)   # remove text in brackets
    t = re.sub(r'\w+', '', t, flags = re.ASCII)   # remove digits and ASCII letters
    t = re.sub(r"\W[^\w\s’]+", '', t)
    t = re.sub(r'\s\w\W\s', ' ', ' '+ t +' ')
    t = re.sub(r'\s+', ' ', t.strip())   # replace all multiple blank charachters with single space
    if len(t) < 32:
      continue
    if check_cyrillic(t):
      texts.append(t)
  return texts

dataset = []
for k,v in lang_data.items():
  lang = v['wiki']
  links = hrefs[lang]
  wikipedia.set_lang(lang)
  wikipedia.set_rate_limiting(True)
  texts = []
  print(lang, end=' ')
  time.sleep(.05)
  for link in links:
    texts.extend(handle_link(link))
    print('=', end='')
    if len(texts) > 1024:
      break
  print(f" {len(texts)} ", end='')

  while len(texts) < 40:
    for page in wikipedia.random(10):
      try:
        p = wikipedia.page(page)
        texts.extend(handle_link(p.url))
      except wikipedia.exceptions.DisambiguationError:
        pass
    print(f"{ len(texts)} ", end='')
  for t in set(texts):      # remove duplicates
    dataset.append((t, k))
  print("@")

ab ========================================== 37 45 @
ady =============================== 82 @
alt =========== 380 @
av ================================================ 198 @
ba ================================== 1043 @
be ================================= 1031 @
bg ====================== 1053 @
bxr ============================================================================== 574 @
ce ==================================================================== 806 @
cv ================================================================================= 71 @
mhr =========================================================== 252 @
myv ============================================================== 215 @
inh ================================================================= 146 @
kbd ============================= 245 @
xal ======================================== 122 @
krc =============================================== 431 @
kk =============================== 1037 @
kv =================================

/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


6 7 8 9 10 14 14 14 14 15 16 16 17 17 19 21 23 23 23 24 24 24 24 24 24 26 27 29 29 30 30 31 31 32 34 35 39 40 @
ru ============= 1118 @
rue =============================================================================== 202 @
sr ===================== 1033 @
tg ============================================================================== 537 @
tt ============================================================================ 1057 @
tyv ========================= 137 @
udm ================================================= 93 @
uk ================== 1090 @
mrj ================================= 62 @
sah ================================================================================== 490 @


In [49]:
# get texts from Tatoeba

for k,v in lang_data.items():
  lang = v['tato']
  for sent in tatoeba.sentences_detailed(lang):
    if len(sent) < 48:
      continue
    dataset.append([sent.text, k])
  

In [50]:
# get texts from UDHR
for k,v in lang_data.items():
  lang = v['udhr']
  if not lang:
    continue
  url = f'https://www.unicode.org/udhr/d/udhr_{lang}.html'
  article = requests.get(url)
  soup = BeautifulSoup(article.text, 'html.parser')
  contents = soup.select("div p")
  for p in contents:
    text = p.get_text()
    t = re.sub(r'\w+', '', text, flags = re.ASCII)   # remove digits and ASCII letters
    t = re.sub(r"\W[^\w\s’]+", '', t)
    t = re.sub(r'\s\w\W\s', ' ', ' '+ t +' ')
    t = re.sub(r'\s+', ' ', t.strip())   # replace all multiple blank charachters with single space
    if t:
      dataset.append([t, k])

In [61]:
dataset_cleared = []
for item in dataset:
  text, lang = item[0], item[1]
  ts = re.split(r'[\W\d]+', text.lower()) 
  dataset_cleared.append([' '.join([w for w in ts if check_cyrillic(w)]).strip(), lang])





In [62]:
df = pd.DataFrame.from_records(dataset_cleared, columns=['text', 'lang'])

In [66]:
df.to_csv('dataset_cleared.csv', sep='\t')

In [54]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [64]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['lang'], random_state=2023)

vec = CountVectorizer(ngram_range=(1, 2), analyzer='char_wb', max_df=0.5, min_df=0.001)
X_train_vectorized = vec.fit_transform(X_train)
X_test_vectorized = vec.transform(X_test)

In [65]:
vec.get_feature_names_out()

array([' а', ' б', ' в', ' г', ' д', ' е', ' ж', ' з', ' и', ' й', ' к',
       ' л', ' м', ' н', ' о', ' п', ' р', ' с', ' т', ' у', ' ф', ' х',
       ' ц', ' ч', ' ш', ' щ', ' ы', ' э', ' ю', ' я', ' ё', ' є', ' і',
       ' ї', ' ј', ' ќ', ' ў', ' җ', ' қ', ' ү', ' һ', ' ә', ' ө', 'а ',
       'аа', 'аб', 'ав', 'аг', 'ад', 'ае', 'аж', 'аз', 'аи', 'ай', 'ак',
       'ал', 'ам', 'ан', 'ао', 'ап', 'ар', 'ас', 'ат', 'ау', 'аф', 'ах',
       'ац', 'ач', 'аш', 'ащ', 'аю', 'ая', 'аё', 'ає', 'ај', 'ањ', 'аў',
       'ағ', 'ақ', 'аң', 'б', 'б ', 'ба', 'бв', 'бе', 'би', 'бк', 'бл',
       'бм', 'бн', 'бо', 'бр', 'бс', 'бу', 'бх', 'бщ', 'бъ', 'бы', 'бя',
       'бё', 'бі', 'бү', 'бә', 'бө', 'в ', 'ва', 'вв', 'вд', 'ве', 'вж',
       'вз', 'ви', 'вк', 'вл', 'вм', 'вн', 'во', 'вп', 'вр', 'вс', 'вт',
       'ву', 'вч', 'вш', 'въ', 'вы', 'вь', 'вя', 'вё', 'ві', 'г', 'г ',
       'га', 'гд', 'ге', 'ги', 'гк', 'гл', 'гн', 'го', 'гр', 'гт', 'гу',
       'гъ', 'гы', 'гэ', 'гі', 'гү', 'гә', 'д ', 'да'

In [57]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
clf = LogisticRegression(max_iter=500).fit(X_train_vectorized, y_train)
preds = clf.predict(X_test_vectorized)
print(classification_report(y_test, preds))


KeyboardInterrupt: ignored

In [ ]:
pd.DataFrame(X_train_vectorized.toarray(), columns=vec.get_feature_names_out()).head()

,"""",-,.,«,²,³,¹,е,ж,з,...,’е,’я,“,“,”,”,•,•,…,…
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,11,20,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0,2,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
X = df['text']
y = df['lang']
X_vec = vec.transform(X)
preds = clf.predict(X_vec)

In [ ]:
res = pd.DataFrame({'txt':X, 'lng':y, 'pred':preds})

In [ ]:
res.loc[res['lng'] != res['pred']]

,txt,lng,pred
0,Сара азба сакуеит.,ab,tt
1,Сара ашә сфоит.,ab,tt
2,Шьыжьбзиа!,ab,mhr
3,Сара акәац сфоит.,ab,tt
4,Иуфазеи?,ab,mhr
...,...,...,...
1370516,"Ик дала уснд хандго, ик мергн эрдмд хандго.",xal,mn
1370517,"Ирхинчн седкл бәәхлә, иҗлинчн усн чальчаг.",xal,tt
1370518,Бурдж Халифа ода деернь делкәнд хамгин өндр бә...,xal,tt
1370519,Нарт-делкән күүкд улсин өдрлә йөрәҗәнәв!,xal,tt


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(df['text'], df['lang'], random_state=2023)

vec = TfidfVectorizer(ngram_range=(1, 2), analyzer='char_wb', max_df=0.5, min_df=0.01)
X_train_vectorized = vec.fit_transform(X_train)
X_test_vectorized = vec.transform(X_test)

In [ ]:
vec.get_feature_names_out()

array([' е', ' ж', ' з', ..., '’', '“', '“ '], dtype=object)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from matplotlib import pyplot as plt
from sklearn import tree

#clf = LogisticRegression().fit(X_train_vectorized, y_train)
preds = clf.predict(X_test_vectorized)
print(classification_report(y_test, preds))
#tree.plot_tree(clf)
#plt.show()

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

              precision    recall  f1-score   support

          ab       0.09      0.14      0.11        28
         ady       0.50      0.16      0.24        31
         alt       0.03      0.38      0.05         8
          av       0.01      1.00      0.01         9
          ba       0.19      0.80      0.30       252
          be       0.67      0.82      0.74     13392
          bg       0.20      0.62      0.31     24861
         bxr       0.40      0.63      0.49       906
          ce       0.00      0.88      0.01        40
          cu       0.00      0.00      0.00         0
          cv       0.99      0.45      0.62      1658
         inh       0.00      0.00      0.00         0
         kbd       0.53      0.83      0.65        87
          kk       0.82      0.89      0.85      4154
         koi       0.12      0.51      0.19        43
         kpv       0.00      0.00      0.00        46
         krc       0.01      0.32      0.01       210
          kv       0.00    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
pd.DataFrame(X_train_vectorized.toarray(), columns=vec.get_feature_names_out())

,е,ж,з,й,л,р,у,ф,х,ц,...,өч,өш,өң,өө,ӯ,ӱ,ӱн,’,“,“
0,0.000000,0.000000,0.000000,0.0,0.076576,0.121314,0.182353,0.000000,0.068369,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.000000,0.0,0.000000,0.017469,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,0.055652,0.121625,0.000000,0.0,0.031296,0.012395,0.045543,0.027991,0.009314,0.012224,...,0.0,0.0,0.0,0.009326,0.0,0.0,0.0,0.0,0.0,0.0
3,0.047061,0.000000,0.079351,0.0,0.000000,0.038433,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.0,0.000000,0.022445,0.000000,0.030412,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12436,0.000000,0.000000,0.000000,0.0,0.123762,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
12437,0.159755,0.064009,0.067341,0.0,0.054901,0.108720,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
12438,0.000000,0.090377,0.042259,0.0,0.000000,0.143274,0.000000,0.000000,0.115349,0.030278,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
12439,0.000000,0.144463,0.000000,0.0,0.000000,0.000000,0.000000,0.066493,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
!pip install tensorflow_text
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import utils
from tensorflow.keras.layers import TextVectorization

import tensorflow_datasets as tfds
import tensorflow_text as tf_text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 64.7 MB/s eta 0:00:00


In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

vectorize_layer = tf.keras.layers.TextVectorization(
 standardize='lower_and_strip_punctuation',
 split='character',
 ngrams=2,
 output_mode='count',
)

In [ ]:
BUFFER_SIZE = 2048
BATCH_SIZE = 16

train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
vectorize_layer.adapt(train_dataset.map(lambda text, label: text))

In [ ]:
model = tf.keras.Sequential([
    vectorize_layer,
    tf.keras.layers.Embedding(
        input_dim=len(vectorize_layer.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(37, activation='relu'),
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_5 (TextV  (None, 5795)             0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 5795, 64)          370880    
                                                                 
 bidirectional (Bidirectiona  (None, 128)              66048     
 l)                                                              
                                                                 
 dense (Dense)               (None, 37)                4773      
                                                                 
Total params: 441,701
Trainable params: 441,701
Non-trainable params: 0
_________________________________________________________________
